# オンラインエンドポイントへのデプロイ

アプリケーションからモデルを利用するには、オンラインエンドポイントにモデルをデプロイします。ローカルファイルからMLflowモデルを作成し、エンドポイントをテストします。

## 始める前に

このノートブックのコードを実行するには、**azure-ai-ml** パッケージの最新版が必要です。以下のセルを実行して、インストールされていることを確認してください。

> 注**：
> もし **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールしてください。

In [ ]:
pip show azure-ai-ml

## ワークスペースに接続する

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、サブスクリプションID、リソースグループ名、ワークスペース名といった識別子のパラメータが必要だ。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、デフォルト値を使用してワークスペースに接続できる。

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

## エンドポイントの定義と作成
最終的なゴールは、エンドポイントにモデルをデプロイすることです。そのため、まずエンドポイントを作成する必要があります。エンドポイントは、アプリケーションがモデルから予測を受け取るために呼び出すことができるHTTPSエンドポイントになります。アプリケーションは、そのURIを使用し、キーまたはトークンで認証することで、エンドポイントを利用することができます。

次のセルを実行してエンドポイントを定義します。エンドポイントの名前は一意でなければならないことに注意してください。一意な名前を生成するには `datetime` 関数を使用します。

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MLflow diabetes model",
    auth_mode="key",
)

次に、以下のセルを実行してエンドポイントを作成する。これには数分かかるかもしれない。エンドポイントが作成されている間に、[Azure Machine Learning エンドポイントとは](https://learn.microsoft.com/azure/machine-learning/concept-endpoints) を読むことができます。

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

<p style=「color:red;font-size:120%;background-color:yellow;font-weight:bold」> 重要です！続行する前に、エンドポイントが正常に作成されるまで待ちます！スタジオに緑色の通知が表示されるはずです。</p>

## デプロイの設定

1つのエンドポイントに複数のモデルをデプロイできます。これは、現在のモデルを本番環境に置いたまま、デプロイされたモデルを更新したい場合に便利です。どのモデルをエンドポイントにデプロイするかを指定するために、デプロイメントを設定する必要があります。以下のセルでは、ローカルの `model` フォルダ（このノートブックと同じフォルダに格納されています）に格納されている学習済みのモデルを参照します。MLflow モデルで作業しているので、環境やスコアリングスクリプトを指定する必要はないことに注意してください。

モデルのデプロイに必要なインフラストラクチャも指定します。

In [ ]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# create a blue deployment
model = Model(
    path="./model",
    type=AssetTypes.MLFLOW_MODEL,
    description="my sample mlflow model",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_D2as_v4",
    instance_count=1,
)

## Create the deployment

Finally, you can actually deploy the model to the endpoint by running the following cell:

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

モデルのデプロイには10～15分かかる場合があります。モデルがデプロイされるのを待つ間、[このビデオの管理対象エンドポイント](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper) について詳しく学ぶことができます。

<p style=「color:red;font-size:120%;background-color:yellow;font-weight:bold」> 重要です！デプロイが完了するまで待ってから続行してください！スタジオに緑色の通知が表示されるはずです</p>。

エンドポイントにデプロイしたモデルは1つだけなので、このデプロイメントがトラフィックの100%を占めるようにします。エンドポイントに複数のモデルをデプロイする場合、同じアプローチを使用して、デプロイされたモデル間でトラフィックを分散できます。


In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the blue deployment is configured before continuing! A green notification should appear in the studio. </p> 

## Test the deployment

エンドポイントを呼び出して、デプロイされたモデルをテストしてみましょう。サンプルデータを含むJSONファイルが入力として使用される。学習されたモデルは、年齢、BMI、妊娠回数などの医療データに基づいて、患者が糖尿病かどうかを予測します。0]`は患者が糖尿病でないことを示す。1]`は患者が糖尿病であることを意味する。

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

オプションとして、`sample-data.json`ファイルの値を変更して、異なる予測を試みることもできる。

## エンドポイントの一覧表示

スタジオではすべてのエンドポイントを表示できますが、SDKを使用してすべてのエンドポイントを一覧表示することもできます：

In [ ]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

## エンドポイントの詳細を取得する

特定のエンドポイントに関する詳細情報が必要な場合は、SDKを使用して詳細を調べることもできます。

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## エンドポイントとデプロイメントを削除する

エンドポイントは常に利用可能なので、コスト削減のために一時停止することはできません。不必要なコストを避けるには、エンドポイントを削除してください。

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)